In [1]:
import scCube
from scCube import scCube
from scCube.visualization import *
from scCube.utils import *
from pathlib import Path
from matplotlib.pyplot import rc_context
import pandas as pd
import scanpy as sc
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")

In [2]:
model = scCube()

In [4]:
train_sample = '151673'
sc_data = pd.read_csv(f"./DLPFC/{train_sample}/data.csv", index_col = 0)
sc_meta = pd.read_csv(f"./DLPFC/{train_sample}/meta.csv", index_col = 0)
adata = model.pre_process(
    sc_data=sc_data.T, 
    sc_meta=sc_meta,
    is_normalized=False
    )
coor_df = adata.obs[["imagecol","imagerow"]]
adata.obsm["spatial"] = coor_df.to_numpy()
adata.obs['spot'] = adata.obs.index
adata.layers["data"] = adata.X

the input is count matrix, normalizing it firstly...


In [11]:
import random
def generate_list(raw_list, num):
    selected_clusters = random.sample(raw_list, num)
    count_list = []
    free_cluster = len(raw_list) - num
    for i in range(num):
        if i == num - 1:
            random_count = free_cluster
        else:
            random_count = random.randint(0, free_cluster)
        count_list.append(1 + random_count)
        free_cluster -= random_count
    result_list = []
    for cluster, count in zip(selected_clusters, count_list):
        result_list.extend([cluster] * count)    

    return result_list

In [ ]:
for rep in range(1, 11):
    list1 = ['Layer1', 'Layer2', 'Layer3', 'Layer4', 'Layer5', 'Layer6', 'WM']
    list2 = generate_list(list1, 3) # num in [2, 3, 4, 5, 6]
    print("list2:", list2)
    
    test_adata = adata.copy()
    test_adata.obs['sim_truth'] = test_adata.obs['ground_truth']
    for i in range(len(list1)):
        test_adata.obs.loc[test_adata.obs['ground_truth'] == list1[i], 'sim_truth'] = list2[i]

    train_adata = adata.copy()
    drop_cluter = list(set(list1) - set(list2))
    for i in range(len(drop_cluter)):
        train_adata = train_adata[train_adata.obs["ground_truth"] != drop_cluter[i]]    
        
    generate_sc_meta, generate_sc_data = model.load_vae_and_generate_cell(
        sc_adata=train_adata,
        celltype_key='ground_truth',
        cell_key='spot',
        target_num=dict(test_adata.obs.sim_truth.value_counts()),
        hidden_size=128,
        load_path="./DLPFC_151673train.pth",
        used_device='cuda:0')
    generate_sc_data, generate_sc_meta = model.generate_pattern_reference(
            sc_adata=test_adata,
            generate_sc_data=generate_sc_data,
            generate_sc_meta=generate_sc_meta,
            celltype_key='sim_truth',
            spatial_key=['imagecol', 'imagerow'],
            cost_metric='sqeuclidean')
    
    new_adata = sc.AnnData(generate_sc_data.T)
    new_adata.var_names_make_unique()
    new_adata.obs = generate_sc_meta
    coor_df = new_adata.obs[["imagecol","imagerow"]]
    new_adata.obsm["spatial"] = coor_df.to_numpy()
    new_adata.obs["ground_truth"] = new_adata.obs["Cell_type"]
    select_cluter = list(set(list2))
    for i in range(len(select_cluter)):
        new_adata.obs.loc[new_adata.obs['ground_truth'] == select_cluter[i], 'ground_truth'] = str(i)
    sc.pl.embedding(new_adata, basis="spatial", color="ground_truth", show=False)

    new_adata.write_h5ad(f"{save_path}/adata.h5ad")